# Imports

In [ ]:


import pandas as pd
import requests
import os
from tqdm import tqdm
import methylprep

In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np
# from missingpy import MissForest
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
# a dictionary to put all of the accuracies
accuracies = {}

In [ ]:
# loading data with 10% missing
gse111631_10 = pd.read_csv('/content/data/Shareddrives/Deep_Learning_G&R/gse111631_10.csv').rename(columns={'Unnamed: 0':'Sample'}).set_index('Sample')
print(gse111631_10.isna().sum().sum()/(gse111631_10.shape[0]*gse111631_10.shape[1])*100, '% missing values')
print(gse111631_10.shape)
gse111631_10.head()

10.252032520325203 % missing values
(123, 100)


,cg17357548,cg24592462,cg07068870,cg07576175,cg04384867,cg06293982,cg07452706,cg09622586,cg19249622,cg26688435,...,cg01021551,cg13299707,cg23355015,cg26557696,cg00942293,cg23842941,cg17573068,cg11070059,cg27352639,cg01744133
Sample,,,,,,,,,,,,,,,,,,,,,
201172560029_R04C01,0.945,0.014,0.066,0.022,NaN,0.940,0.300,NaN,0.972,0.964,...,0.016,0.858,0.976,0.896,0.870,NaN,0.015,0.159,0.045,0.048
201172560029_R05C01,0.949,0.013,0.048,0.026,0.957,0.930,0.352,0.957,0.973,0.960,...,0.015,0.776,0.973,0.922,0.898,0.736,0.017,0.158,0.063,0.055
201172560029_R06C01,0.932,0.013,0.051,0.020,0.964,0.936,0.315,0.964,0.970,0.963,...,NaN,0.795,0.977,0.930,0.891,0.741,0.016,0.181,0.065,0.043
201172560029_R07C01,0.951,0.011,NaN,0.029,0.967,NaN,0.242,0.957,0.953,0.966,...,0.016,NaN,0.972,0.927,NaN,0.755,0.018,0.159,0.066,0.055
201172560029_R08C01,0.932,0.011,0.062,0.025,0.970,0.939,0.335,0.955,0.969,0.942,...,0.019,0.866,0.979,0.855,0.884,0.788,0.016,0.237,0.043,0.048


In [ ]:
# loading original
original_gse111631 = pd.read_csv('/content/data/Shareddrives/Deep_Learning_G&R/gse111631_original.csv').rename(columns={'Unnamed: 0':'Sample'}).set_index('Sample')
print(original_gse111631.shape)
original_gse111631.head()

(123, 100)


,cg17357548,cg24592462,cg07068870,cg07576175,cg04384867,cg06293982,cg07452706,cg09622586,cg19249622,cg26688435,...,cg01021551,cg13299707,cg23355015,cg26557696,cg00942293,cg23842941,cg17573068,cg11070059,cg27352639,cg01744133
Sample,,,,,,,,,,,,,,,,,,,,,
201172560029_R04C01,0.945,0.014,0.066,0.022,0.973,0.940,0.300,0.929,0.972,0.964,...,0.016,0.858,0.976,0.896,0.870,0.751,0.015,0.159,0.045,0.048
201172560029_R05C01,0.949,0.013,0.048,0.026,0.957,0.930,0.352,0.957,0.973,0.960,...,0.015,0.776,0.973,0.922,0.898,0.736,0.017,0.158,0.063,0.055
201172560029_R06C01,0.932,0.013,0.051,0.020,0.964,0.936,0.315,0.964,0.970,0.963,...,0.015,0.795,0.977,0.930,0.891,0.741,0.016,0.181,0.065,0.043
201172560029_R07C01,0.951,0.011,0.068,0.029,0.967,0.922,0.242,0.957,0.953,0.966,...,0.016,0.836,0.972,0.927,0.880,0.755,0.018,0.159,0.066,0.055
201172560029_R08C01,0.932,0.011,0.062,0.025,0.970,0.939,0.335,0.955,0.969,0.942,...,0.019,0.866,0.979,0.855,0.884,0.788,0.016,0.237,0.043,0.048


In [ ]:
def get_accuracies(imputed, missing=gse111631_10, original=original_gse111631):
    # sum of squared errors, mean of average absolute errors, root mean squared error
    ret = {}
    ret['sse'] = ((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().sum()
    ret['mae'] = (imputed.mask(~missing.isna()) - original.mask(~missing.isna())).abs().sum().mean()
    ret['rmse'] = (((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().mean())**(1/2)
    return ret

**LSTM**

In [ ]:
def create_sequences(data, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length + 1):
        X.append(data[i : i + sequence_length])
        y.append(data[i + sequence_length - 1])
    return np.array(X), np.array(y)


In [ ]:

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming original and missing are pandas DataFrames
def create_sequences(data, sequence_length):
    X = []
    y = []
    for i in range(len(data) - sequence_length):
        X.append(data[i : i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

def lstm_imputation(original, missing):
      # Verify column dimensions
    min_length = min(len(original), len(missing))
    original = original[:min_length]
    missing = missing[:min_length]
    # Preprocess the data
    scaler = MinMaxScaler()
    original_scaled = scaler.fit_transform(original)
    missing_scaled = scaler.transform(missing)

    # Create input sequences and labels
    sequence_length = 10  # Define the sequence length
    X_train, y_train = create_sequences(original_scaled, sequence_length)

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the LSTM model
    model.fit(X_train, y_train, epochs=10, batch_size=16)

    # Perform data imputation
    X_test, y_test = create_sequences(missing_scaled, sequence_length)
    imputed_scaled = model.predict(X_test)

    # Inverse scaling
    imputed = scaler.inverse_transform(np.repeat(imputed_scaled, original.shape[1], axis=1))
    imputed_df = pd.DataFrame(imputed, columns=original.columns)

    # Calculate metrics
    sse = np.sum((original - imputed_df) ** 2)
    mae = mean_absolute_error(original, imputed_df)
    rmse = np.sqrt(mean_squared_error(original, imputed_df))

    return imputed_df, {'sse': sse, 'mae': mae, 'rmse': rmse}




In [ ]:
lstm_df, lstm_accs = lstm_imputation(original_gse111631, gse111631_10)
lstm_accs

Multiple Imputation by Chained Equations (MICE) with deep learning model

In [ ]:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.neural_network import MLPRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

def deep_learning_imputation(original_data, missing_data):
    # Create a copy of the missing data
    imputed_data = missing_data.copy()

    # Identify columns with missing values
    missing_columns = imputed_data.columns[imputed_data.isnull().any()].tolist()

    # Create a separate model for each missing column
    for column in missing_columns:
        # Create a pipeline for preprocessing and imputation
        pipeline = Pipeline([
            ('scaler', MinMaxScaler()),
            ('imputer', IterativeImputer(estimator=MLPRegressor(hidden_layer_sizes=(64, 64)))),
            ('inverse_scaler', MinMaxScaler(feature_range=(original_data[column].min(), original_data[column].max())))
        ])

        # Fit the pipeline to the observed data
        pipeline.fit(original_data[column].values.reshape(-1, 1))

        # Generate imputations for missing values
        imputations = pipeline.transform(imputed_data[column].values.reshape(-1, 1))

        # Replace missing values with imputations
        imputed_data[column] = np.where(imputed_data[column].isnull(), imputations, imputed_data[column])

    return imputed_data




In [ ]:

# Assuming original_gse111631 is the original data without missing values
# and gse111631_10 is the data with 10% missing values
imputed_data = deep_learning_imputation(original_gse111631, gse111631_10)
# Calculate error metrics
mse = mean_squared_error(original_gse111631, imputed_data)
mae = mean_absolute_error(original_gse111631, imputed_data)
rmse = np.sqrt(mse)

# Print the error metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Print the imputed data
print(imputed_data)

Mean Squared Error (MSE): 0.5181035792140921
Mean Absolute Error (MAE): 0.6181160704607046
Root Mean Squared Error (RMSE): 0.7197941227976873
                     cg17357548  cg24592462  cg07068870  cg07576175  \
Sample                                                                
201172560029_R04C01       0.055       0.043       0.055       0.048   
201172560029_R05C01       0.055       0.043       0.055       0.048   
201172560029_R06C01       0.055       0.043       0.055       0.048   
201172560029_R07C01       0.055       0.043       0.055       0.048   
201172560029_R08C01       0.055       0.043       0.055       0.048   
...                         ...         ...         ...         ...   
201533510060_R04C01       0.055       0.043       0.055       0.048   
201533510060_R05C01       0.055       0.043       0.055       0.048   
201533510060_R06C01       0.055       0.043       0.055       0.048   
201533510060_R07C01       0.055       0.043       0.055       0.048   
201533

**GAIN** 

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error

class GAINImputer:
    def __init__(self, data, generator, discriminator, alpha=100, epochs=100, batch_size=128):
        self.data = data.copy()
        self.generator = generator
        self.discriminator = discriminator
        self.alpha = alpha
        self.epochs = epochs
        self.batch_size = batch_size

    def gain_loss_generator(self):
        # Define the loss function for the generator
        def loss_generator(y_true, y_pred):
            observed_mask = K.cast(K.not_equal(y_true, 0), K.floatx())
            mse_loss = K.mean(K.square(observed_mask * (y_true - y_pred)))
            return mse_loss

        return loss_generator

    def gain_loss_discriminator(self):
        # Define the loss function for the discriminator
        def loss_discriminator(y_true, y_pred):
            observed_mask = K.cast(K.not_equal(y_true, 0), K.floatx())
            mse_loss = K.mean(K.square(observed_mask * (y_true - y_pred)))
            return mse_loss

        return loss_discriminator

    def train_gan(self):
        # Define the GAIN loss functions and optimizers
        loss_generator = self.gain_loss_generator()
        loss_discriminator = self.gain_loss_discriminator()
        optimizer_generator = Adam()
        optimizer_discriminator = Adam()

        # Rest of the training code...

    # Rest of the GAINImputer class implementation...

def gain_imputation(data, alpha=100, epochs=100, batch_size=128, threshold=0.5):
    # Create a copy of the data
    imputed_data = data.copy()

    # Identify columns with missing values
    missing_columns = imputed_data.columns[imputed_data.isnull().any()].tolist()

    # Create a separate model for each missing column
    for column in missing_columns:
        # Filter data for the current column
        column_data = imputed_data[[column]].copy()

        # Create the generator and discriminator models
        generator = Sequential()
        generator.add(Dense(128, activation='relu'))
        generator.add(Dense(column_data.shape[1], activation='linear'))

        discriminator = Sequential()
        discriminator.add(Dense(128, activation='relu'))
        discriminator.add(Dense(1, activation='sigmoid'))

        # Create the GAIN imputer
        imputer = GAINImputer(column_data.values, generator, discriminator,
                              alpha=alpha, epochs=epochs, batch_size=batch_size)

        # Perform GAIN imputation for the current column
        imputer.train_gan()

        # Replace missing values with imputations
        imputed_values = column_data.values
        imputed_values[np.abs(imputed_values - column_data.values) < threshold] = np.nan
        imputed_data[column] = imputed_values

    return imputed_data

# Assuming gse111631_10 is the data with 10% missing values
imputed_data = gain_imputation(gse111631_10)

# Calculate error metrics
mse = mean_squared_error(gse111631_10, imputed_data)
mae = mean_absolute_error(gse111631_10, imputed_data)
rmse = np.sqrt(mse)

# Print the error metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", m)

In [ ]:
# Assuming gse111631_10 is the data with 10% missing values
imputed_data = gain_imputation(gse111631_10)

# Print the imputed data
print(imputed_data)

# Assuming original_gse111631 is the original complete data without missing values
# Calculate error metrics
mse = mean_squared_error(original_gse111631, imputed_data)
mae = mean_absolute_error(original_gse111631, imputed_data)
rmse = np.sqrt(mse)

# Print the error metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate the error metrics
mse = mean_squared_error(original_complete_data, imputed_data)
mae = mean_absolute_error(original_complete_data, imputed_data)
rmse = np.sqrt(mse)

# Print the error metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


In [ ]:
has_missing_values = gse111631_10.isnull().any().any()

if has_missing_values:
    print("The DataFrame has missing values.")
else:
    print("The DataFrame does not have any missing values.")


The DataFrame has missing values.


In [ ]:
# Calculate the number of missing values per column
missing_counts = gse111631_10.isnull().sum()

# Calculate the percentage of missing values per column
missing_percentages = (missing_counts / gse111631_10.shape[0]) * 100

# Display the results
for column in missing_counts.index:
    missing_count = missing_counts[column]
    missing_percentage = missing_percentages[column]
    print(f"Column '{column}': Missing Count: {missing_count}, Missing Percentage: {missing_percentage:.2f}%")


Column 'cg17357548': Missing Count: 14, Missing Percentage: 11.38%
Column 'cg24592462': Missing Count: 12, Missing Percentage: 9.76%
Column 'cg07068870': Missing Count: 9, Missing Percentage: 7.32%
Column 'cg07576175': Missing Count: 14, Missing Percentage: 11.38%
Column 'cg04384867': Missing Count: 13, Missing Percentage: 10.57%
Column 'cg06293982': Missing Count: 18, Missing Percentage: 14.63%
Column 'cg07452706': Missing Count: 8, Missing Percentage: 6.50%
Column 'cg09622586': Missing Count: 13, Missing Percentage: 10.57%
Column 'cg19249622': Missing Count: 16, Missing Percentage: 13.01%
Column 'cg26688435': Missing Count: 14, Missing Percentage: 11.38%
Column 'cg15700022': Missing Count: 12, Missing Percentage: 9.76%
Column 'cg01195833': Missing Count: 9, Missing Percentage: 7.32%
Column 'cg25035655': Missing Count: 9, Missing Percentage: 7.32%
Column 'cg22598767': Missing Count: 10, Missing Percentage: 8.13%
Column 'cg01390061': Missing Count: 9, Missing Percentage: 7.32%
Column '

**Deep Gaussian Processes for imputation** Working

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

def dgp_imputation(original_data, missing_data):
    # Create a copy of the missing data
    imputed_data = missing_data.copy()

    # Identify columns with missing values
    missing_columns = imputed_data.columns[imputed_data.isnull().any()].tolist()

    # Create a separate model for each missing column
    for column in missing_columns:
        # Filter data for the current column
        column_data = imputed_data[[column]].copy()

        # Preprocess the data - impute missing values using SimpleImputer and scale using MinMaxScaler
        imputer = SimpleImputer(strategy='mean')
        scaler = MinMaxScaler()
        column_data_imputed_scaled = scaler.fit_transform(imputer.fit_transform(column_data))

        # Apply Gaussian Process Regression to impute missing values
        gp_imputer = GaussianProcessRegressor()
        gp_imputer.fit(column_data_imputed_scaled, column_data_imputed_scaled)

        # Generate imputations for missing values
        imputations_scaled = gp_imputer.sample_y(column_data_imputed_scaled)

        # Inverse scaling
        imputations = scaler.inverse_transform(imputations_scaled)

        # Replace missing values with imputations
        imputed_data[column] = np.where(imputed_data[column].isnull(), imputations, imputed_data[column])

    return imputed_data

# Assuming original_gse111631 is the original data without missing values
# and gse111631_10 is the data with 10% missing values
imputed_data = dgp_imputation(original_gse111631, gse111631_10)

# Calculate error metrics
mse = mean_squared_error(original_gse111631, imputed_data)
mae = mean_absolute_error(original_gse111631, imputed_data)
rmse = np.sqrt(mse)

# Print the error metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Print the imputed data
print(imputed_data)


Mean Squared Error (MSE): 0.002167756776548099
Mean Absolute Error (MAE): 0.023550243958179253
Root Mean Squared Error (RMSE): 0.0465591749985768
                     cg17357548  cg24592462  cg07068870  cg07576175  \
Sample                                                                
201172560029_R04C01       0.945       0.014       0.066       0.022   
201172560029_R05C01       0.945       0.014       0.066       0.022   
201172560029_R06C01       0.945       0.014       0.066       0.022   
201172560029_R07C01       0.945       0.014       0.066       0.022   
201172560029_R08C01       0.945       0.014       0.066       0.022   
...                         ...         ...         ...         ...   
201533510060_R04C01       0.945       0.014       0.066       0.022   
201533510060_R05C01       0.945       0.014       0.066       0.022   
201533510060_R06C01       0.945       0.014       0.066       0.022   
201533510060_R07C01       0.945       0.014       0.066       0.022   
20

Matrix

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, mean_absolute_error

def matrix_factorization_imputation(original_data, missing_data):
    # Create a copy of the missing data
    imputed_data = missing_data.copy()

    # Identify columns with missing values
    missing_columns = imputed_data.columns[imputed_data.isnull().any()].tolist()

    # Create dictionaries to store error metrics for each column
    mse_scores = {}
    mae_scores = {}

    # Create a separate model for each missing column
    for column in missing_columns:
        # Filter data for the current column
        column_data = imputed_data[[column]].copy()

        # Preprocess the data - impute missing values using SimpleImputer and scale using MinMaxScaler
        imputer = SimpleImputer(strategy='mean')
        scaler = MinMaxScaler()
        column_data_imputed_scaled = scaler.fit_transform(imputer.fit_transform(column_data))

        # Apply Matrix Factorization to impute missing values
        svd_imputer = TruncatedSVD(n_components=n_components)
        column_data_imputed_scaled_svd = svd_imputer.fit_transform(column_data_imputed_scaled)

        # Inverse scaling
        imputations_scaled = svd_imputer.inverse_transform(column_data_imputed_scaled_svd)
        imputations = scaler.inverse_transform(imputations_scaled)

        # Replace missing values with imputations
        imputed_data[column] = np.where(imputed_data[column].isnull(), imputations, imputed_data[column])

        # Calculate error metrics for the imputed column
        original_column_data = original_data[column].values
        imputed_column_data = imputed_data[column].values

        mse = mean_squared_error(original_column_data, imputed_column_data)
        mae = mean_absolute_error(original_column_data, imputed_column_data)

        mse_scores[column] = mse
        mae_scores[column] = mae

    return imputed_data, mse_scores, mae_scores


In [ ]:
# Assuming you already have the original_data and missing_data as Pandas DataFrames

# Perform Matrix Factorization imputation
imputed_data, mse_scores, mae_scores = matrix_factorization_imputation(original_gse111631, gse111631_10)

# Print the imputed data
print(imputed_data)

# Print the MSE scores for each column
print(mse_scores)

# Print the MAE scores for each column
print(mae_scores)



In [ ]:
gse111631_10
print(gse111631_10.shape)

(123, 100)


In [ ]:
missing_20 = pd.read_csv('/content/data/Shareddrives/Deep_Learning_G&R/gse111631_20.csv').rename(columns={'Unnamed: 0':'id'}).set_index('id')
print(missing_20.shape)
missing_20.head()


(123, 100)


,cg17357548,cg24592462,cg07068870,cg07576175,cg04384867,cg06293982,cg07452706,cg09622586,cg19249622,cg26688435,...,cg01021551,cg13299707,cg23355015,cg26557696,cg00942293,cg23842941,cg17573068,cg11070059,cg27352639,cg01744133
id,,,,,,,,,,,,,,,,,,,,,
201172560029_R04C01,0.945,0.014,0.066,NaN,0.973,0.940,0.300,0.929,0.972,0.964,...,0.016,NaN,NaN,0.896,0.870,0.751,0.015,0.159,0.045,0.048
201172560029_R05C01,0.949,0.013,0.048,0.026,0.957,0.930,0.352,0.957,0.973,0.960,...,0.015,0.776,0.973,0.922,NaN,0.736,0.017,NaN,0.063,NaN
201172560029_R06C01,0.932,0.013,0.051,0.020,0.964,0.936,0.315,0.964,0.970,0.963,...,0.015,0.795,0.977,0.930,0.891,0.741,0.016,0.181,NaN,NaN
201172560029_R07C01,0.951,0.011,0.068,0.029,0.967,0.922,0.242,NaN,0.953,0.966,...,NaN,0.836,0.972,0.927,NaN,NaN,0.018,0.159,0.066,0.055
201172560029_R08C01,0.932,0.011,NaN,0.025,0.970,0.939,0.335,0.955,0.969,NaN,...,NaN,0.866,0.979,0.855,0.884,NaN,0.016,0.237,NaN,0.048


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import pandas as pd

def truncated_svd_imputation(original: pd.DataFrame, missing: pd.DataFrame, n_components: int, n_iter: int):
    print('Starting Truncated SVD imputation')

    # Initial mean imputation
    initial_imputer = SimpleImputer(strategy='mean')
    initial_imputed = pd.DataFrame(initial_imputer.fit_transform(missing), columns=original.columns, index=original.index)

    for _ in range(n_iter):
        # Truncated SVD
        svd = TruncatedSVD(n_components=n_components)
        u = svd.fit_transform(initial_imputed)
        vt = svd.components_

        # Reconstruction
        initial_imputed = pd.DataFrame(np.dot(u, vt), columns=original.columns, index=original.index)

    accuracy = get_accuracies(initial_imputed, missing, original)
    return accuracy

def get_accuracies(imputed, missing, original):
    ret = {}
    ret['sse'] = ((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))**2).sum().sum()
    ret['mae'] = (imputed.mask(~missing.isna()) - original.mask(~missing.isna())).abs().sum().mean()
    ret['rmse'] = (((imputed.mask(~missing.isna()) - original.mask(~missing.isna()))*2).sum().mean())*(1/2)
    return ret

# Assuming original_gse111631 and gse111631_10 are defined
tsvd_accs = truncated_svd_imputation(original_gse111631, missing_20, n_components=50, n_iter=10)
print(tsvd_accs)

Starting Truncated SVD imputation
{'sse': 4.323004349860327, 'mae': 0.5246168505908964, 'rmse': -0.050780110492686194}


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

def transformer_imputation(original_data, missing_data):
    # Create a copy of the missing data
    imputed_data = missing_data.copy()

    # Identify columns with missing values
    missing_columns = imputed_data.columns[imputed_data.isnull().any()].tolist()

    # Create a separate imputer for each missing column
    imputers = {}
    for column in missing_columns:
        # Filter data for the current column
        column_data = imputed_data[[column]].copy()

        # Create a pipeline for imputation and scaling
        pipeline = Pipeline([
            ('imputer', IterativeImputer()),
            ('scaler', MinMaxScaler())
        ])

        # Fit and transform the data for imputation
        imputed_scaled = pipeline.fit_transform(column_data)

        # Inverse scaling
        imputed = pipeline.named_steps['scaler'].inverse_transform(imputed_scaled)

        # Store the imputer for later use
        imputers[column] = pipeline.named_steps['imputer']

        # Replace missing values with imputations
        imputed_data[column] = np.where(imputed_data[column].isnull(), imputed, imputed_data[column])

    return imputed_data, imputers

# Assuming original_gse111631 is the original data without missing values
# and gse111631_10 is the data with 10% missing values
imputed_data, imputers = transformer_imputation(original_gse111631, gse111631_10)

# Calculate error metrics
mse = mean_squared_error(original_gse111631, imputed_data)
mae = mean_absolute_error(original_gse111631, imputed_data)
rmse = np.sqrt(mse)

# Print the error metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

# Print the imputed data
print(imputed_data)


Mean Squared Error (MSE): 0.0021677567625656767
Mean Absolute Error (MAE): 0.02355020544670378
Root Mean Squared Error (RMSE): 0.04655917484841926
                     cg17357548  cg24592462  cg07068870  cg07576175  \
Sample                                                                
201172560029_R04C01       0.945       0.014       0.066       0.022   
201172560029_R05C01       0.945       0.014       0.066       0.022   
201172560029_R06C01       0.945       0.014       0.066       0.022   
201172560029_R07C01       0.945       0.014       0.066       0.022   
201172560029_R08C01       0.945       0.014       0.066       0.022   
...                         ...         ...         ...         ...   
201533510060_R04C01       0.945       0.014       0.066       0.022   
201533510060_R05C01       0.945       0.014       0.066       0.022   
201533510060_R06C01       0.945       0.014       0.066       0.022   
201533510060_R07C01       0.945       0.014       0.066       0.022   
2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
import dgl.function as fn
from torch.utils.data import DataLoader
from dgl.nn import GraphConv

# Step 1: Prepare Graph Data

# Prepare your data as a graph structure using DGL's graph objects (e.g., dgl.DGLGraph)
# Define node features, adjacency matrix, and target variable for each instance
# Split the data into train and test sets if necessary

# Step 2: Define the GNN Architecture

class GNNImputer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNImputer, self).__init__()
        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, output_dim)

    def forward(self, g, features):
        x = self.conv1(g, features)
        x = torch.relu(x)
        x = self.conv2(g, x)
        return x

# Step 3: Train the GNN Model

def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()

    for epoch in range(num_epochs):
        for batched_graph, features, targets in train_loader:
            optimizer.zero_grad()
            output = model(batched_graph, features)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

# Step 4: Impute Missing Values

def impute_missing_values(model, g, features):
    model.eval()
    with torch.no_grad():
        imputed_values = model(g, features)
    # Retrieve the imputed values from the GNN output

# Step 5: Evaluate Imputation Quality (optional)

# Evaluate the imputed values against the ground truth if available
# Measure imputation accuracy, calculate metrics such as MSE, MAE, etc.

# Further customization and implementation is required based on your specific needs
# This code outline serves as a starting point for developing GNN-based imputation using DGL


# Additional helper functions may be needed to preprocess data, handle missing values,
# split data into train/test sets, and perform data transformations specific to GNN-based imputation.


In [ ]:
# Instantiate the GNN model
model = GNNImputer(5, 5, 5)

# Define the loss criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the GNN model
train(model, train_loader, criterion, optimizer, num_epochs)

# Evaluate the imputation quality
mse = evaluate_imputation(model, test_data, test_ground_truth)
print("Mean Squared Error (MSE):", mse)



In [ ]:
pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 24.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class AttentionImputer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout):
        super(AttentionImputer, self).__init__()

        # Input layer
        self.input_layer = nn.Linear(input_dim, hidden_dim)

        # Transformer Encoder layers
        self.transformer_encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim * 4, dropout=dropout)
            for _ in range(num_layers)
        ])

        # Output layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, mask):
        x = self.input_layer(x)

        # Masked attention
        mask = mask.unsqueeze(1).unsqueeze(2)
        x = x.masked_fill(mask == 0, 0)

        # Transformer Encoder layers
        for encoder_layer in self.transformer_encoder_layers:
            x = encoder_layer(x)

        # Output layer
        x = self.output_layer(x)

        return x

# Instantiate the AttentionImputer model
# Set hyperparameters
input_dim = gse111631_10.shape[1]
hidden_dim = 256
output_dim = original_gse111631.shape[1]
num_heads = 8
num_layers = 6
learning_rate = 0.001
batch_size = 32
num_epochs = 100
dropout= 10

imputer = AttentionImputer(input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout)

# Define loss criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(imputer.parameters(), lr=learning_rate)

# Training loop
imputer.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    imputed_data = imputer(input_data, mask)
    loss = criterion(imputed_data, target_data)
    loss.backward()
    optimizer.step()

# Evaluation
imputer.eval()

with torch.no_grad():
    imputed_data = imputer(gse111631_10, mask)
mse = criterion(imputed_data, original_gse111631, gse111631_10)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class AttentionImputer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout):
        super(AttentionImputer, self).__init__()

        # Input layer
        self.input_layer = nn.Linear(input_dim, hidden_dim)

        # Transformer Encoder layers
        # Set hyperparameters
        input_dim = gse111631_10.shape[1]
        hidden_dim = 256
        output_dim = original_gse111631.shape[1]
        num_heads = 8
        num_layers = 6
        learning_rate = 0.001
        batch_size = 32
        num_epochs = 100
        dropout= 10

        self.transformer_encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim * 4, dropout)
            for _ in range(num_layers)
        ])

        # Output layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, mask):
        x = self.input_layer(x)

        # Masked attention
        mask = mask.unsqueeze(1).unsqueeze(2)
        x = x.masked_fill(mask == 0, 0)

        # Transformer Encoder layers
        for encoder_layer in self.transformer_encoder_layers:
            x = encoder_layer(x)

        # Output layer
        x = self.output_layer(x)

        return x

# Instantiate the AttentionImputer model
imputer = AttentionImputer(input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout)

# Define loss criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(imputer.parameters(), lr=learning_rate)

# Training loop
imputer.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    imputed_data = imputer(input_data, mask)
    loss = criterion(imputed_data, target_data)
    loss.backward()
    optimizer.step()

# Evaluation
# Set hyperparameters
input_dim = gse111631_10.shape[1]
hidden_dim = 256
output_dim = original_gse111631.shape[1]
num_heads = 8
num_layers = 6
learning_rate = 0.001
batch_size = 32
num_epochs = 100
dropout= 10

imputer.eval()
with torch.no_grad():
    imputed_data = imputer(input_data, mask)
mse = criterion(imputed_data, target_data)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class MaskedLinear(nn.Linear):
    def __init__(self, in_features, out_features, mask):
        super(MaskedLinear, self).__init__(in_features, out_features)
        self.register_buffer('mask', mask)

    def forward(self, x):
        return F.linear(x, self.weight * self.mask, self.bias)

class MADE(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super(MADE, self).__init__()

        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.num_masks = input_dim

        masks = self.create_masks()

        self.layers = nn.ModuleList()
        in_features = input_dim
        for i, out_features in enumerate(hidden_dims):
            mask = masks[i]
            layer = MaskedLinear(in_features, out_features, mask)
            self.layers.append(layer)
            in_features = out_features

        self.output_layer = MaskedLinear(in_features, input_dim, masks[-1])

    def create_masks(self):
        masks = []
        input_degrees = torch.arange(self.input_dim) % self.input_dim
        hidden_degrees = [torch.arange(dim) % self.input_dim for dim in self.hidden_dims]

        for input_degree, hidden_degree in zip([input_degrees] + hidden_degrees, hidden_degrees + [input_degrees]):
            mask = (hidden_degree.unsqueeze(-1) >= input_degree.unsqueeze(0)).float()
            masks.append(mask)

        return masks

    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))
        x = self.output_layer(x)
        return x

# Instantiate the MADE model
input_dims = gse111631_10.shape[1]
hidden_dims = 256
made = MADE(input_dim, hidden_dims)

# Define loss criterion and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(made.parameters(), lr=0.01)

# Training loop
made.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = made(input_data)
    loss = criterion(output, input_data)
    loss.backward()
    optimizer.step()

# Evaluation
made.eval()
with torch.no_grad():
    output = made(input_data)
mse = criterion(output, input_data)
